In [2]:
import xarray as xr

In [3]:
icePoints = xr.open_dataset("data/jan2013.nc")

In [4]:
print(icePoints)

<xarray.Dataset>
Dimensions:  (YC: 588, XC: 2160, time: 30)
Coordinates:
    Depth    (YC, XC) float32 ...
  * XC       (XC) float32 0.08333 0.25 0.4167 0.5833 ... 359.4 359.6 359.8 359.9
  * YC       (YC) float32 -77.98 -77.95 -77.91 -77.88 ... -30.02 -29.87 -29.72
    iter     (time) int64 ...
    rA       (YC, XC) float32 ...
  * time     (time) datetime64[ns] 2013-01-01 2013-01-02 ... 2013-01-30
Data variables:
    SIarea   (time, YC, XC) float32 ...
Attributes:
    history:  Tue Apr 12 16:00:08 2022: ncks -d time,0,29 bsose_i122_2013to20...
    NCO:      netCDF Operators version 4.9.3-alpha02 (Homepage = http://nco.s...


In [5]:
icePointsDF = icePoints.to_dataframe()

In [19]:
print(icePointsDF)

                                        Depth  SIarea  iter           rA
YC         XC         time                                              
-77.982651 0.083333   2013-01-01     0.000000     0.0  1860   14884064.0
                      2013-01-02     0.000000     0.0  1920   14884064.0
                      2013-01-03     0.000000     0.0  1980   14884064.0
                      2013-01-04     0.000000     0.0  2040   14884064.0
                      2013-01-05     0.000000     0.0  2100   14884064.0
...                                       ...     ...   ...          ...
-29.715317 359.916687 2013-01-26  4736.891602     0.0  3360  292099872.0
                      2013-01-27  4736.891602     0.0  3420  292099872.0
                      2013-01-28  4736.891602     0.0  3480  292099872.0
                      2013-01-29  4736.891602     0.0  3540  292099872.0
                      2013-01-30  4736.891602     0.0  3600  292099872.0

[38102400 rows x 4 columns]


In [6]:
icePointw1 = icePoints.sel(time=slice('2013-01-01','2013-01-07'))
print(icePointw1)

<xarray.Dataset>
Dimensions:  (YC: 588, XC: 2160, time: 7)
Coordinates:
    Depth    (YC, XC) float32 0.0 0.0 0.0 0.0 ... 4.8e+03 4.8e+03 4.737e+03
  * XC       (XC) float32 0.08333 0.25 0.4167 0.5833 ... 359.4 359.6 359.8 359.9
  * YC       (YC) float32 -77.98 -77.95 -77.91 -77.88 ... -30.02 -29.87 -29.72
    iter     (time) int64 1860 1920 1980 2040 2100 2160 2220
    rA       (YC, XC) float32 1.488e+07 1.488e+07 ... 2.921e+08 2.921e+08
  * time     (time) datetime64[ns] 2013-01-01 2013-01-02 ... 2013-01-07
Data variables:
    SIarea   (time, YC, XC) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    history:  Tue Apr 12 16:00:08 2022: ncks -d time,0,29 bsose_i122_2013to20...
    NCO:      netCDF Operators version 4.9.3-alpha02 (Homepage = http://nco.s...


In [7]:
latMin = -74.375
latMax = -73.75

longMin = -130
longMax = -128.75

if longMin < 0: longMin += 360 # converting between projections
if longMax < 0: longMax += 360

In [8]:
%%time

ipSlice = icePointsDF.query("YC > @latMin & YC <= @latMax & XC > @longMin & XC <= @longMax")

ipSlice = ipSlice.reset_index()
ipSlice = ipSlice.rename(columns={'XC':'long', 'YC':'lat', 'Depth':'depth', 'SIarea':'iceArea'})
ipSlice['long'] = ipSlice['long'].apply(lambda x: x if x <= 180 else x - 360)
    
print(ipSlice)

            lat        long       time  depth   iceArea  iter          rA
0    -74.335640 -129.916656 2013-01-01  650.0  0.013741  1860  25030016.0
1    -74.335640 -129.916656 2013-01-02  650.0  0.000000  1920  25030016.0
2    -74.335640 -129.916656 2013-01-03  650.0  0.000000  1980  25030016.0
3    -74.335640 -129.916656 2013-01-04  650.0  0.000000  2040  25030016.0
4    -74.335640 -129.916656 2013-01-05  650.0  0.000154  2100  25030016.0
...         ...         ...        ...    ...       ...   ...         ...
3115 -73.786469 -128.750000 2013-01-26  700.0  0.307928  3360  26767922.0
3116 -73.786469 -128.750000 2013-01-27  700.0  0.295074  3420  26767922.0
3117 -73.786469 -128.750000 2013-01-28  700.0  0.284799  3480  26767922.0
3118 -73.786469 -128.750000 2013-01-29  700.0  0.275683  3540  26767922.0
3119 -73.786469 -128.750000 2013-01-30  700.0  0.271569  3600  26767922.0

[3120 rows x 7 columns]
CPU times: total: 891 ms
Wall time: 885 ms


In [9]:
%%time

icePointCut = icePoints.sel(YC=slice(latMin,latMax), XC=slice(longMin, longMax))
#icePointCut = icePointCut.sel()
ipSlice = icePointCut.to_dataframe()

ipSlice = ipSlice.reset_index()
ipSlice = ipSlice.rename(columns={'XC':'long', 'YC':'lat', 'Depth':'depth', 'SIarea':'iceArea'})
ipSlice['long'] = ipSlice['long'].apply(lambda x: x if x <= 180 else x - 360)

print(ipSlice)

            lat        long       time  depth   iceArea  iter          rA
0    -74.335640 -129.916656 2013-01-01  650.0  0.013741  1860  25030016.0
1    -74.335640 -129.916656 2013-01-02  650.0  0.000000  1920  25030016.0
2    -74.335640 -129.916656 2013-01-03  650.0  0.000000  1980  25030016.0
3    -74.335640 -129.916656 2013-01-04  650.0  0.000000  2040  25030016.0
4    -74.335640 -129.916656 2013-01-05  650.0  0.000154  2100  25030016.0
...         ...         ...        ...    ...       ...   ...         ...
3115 -73.786469 -128.750000 2013-01-26  700.0  0.307928  3360  26767922.0
3116 -73.786469 -128.750000 2013-01-27  700.0  0.295074  3420  26767922.0
3117 -73.786469 -128.750000 2013-01-28  700.0  0.284799  3480  26767922.0
3118 -73.786469 -128.750000 2013-01-29  700.0  0.275683  3540  26767922.0
3119 -73.786469 -128.750000 2013-01-30  700.0  0.271569  3600  26767922.0

[3120 rows x 7 columns]
CPU times: total: 15.6 ms
Wall time: 10.8 ms


In [14]:
latMin = -74.375
latMax = -73.75

longMin = -118.75
longMax = -117.5

if longMin < 0: longMin += 360 # converting between projections
if longMax < 0: longMax += 360

In [ ]:
cellBox = icePointsDF.query("YC > @latMin & YC <= @latMax & XC > @longMin & XC <= @longMax")
print(cellBox)
print(cellBox['SIarea'].mean())

In [ ]:
splittingPercentage = 0.12
splitMinProp = 0.05
splitMaxProp = 0.85

propOver = XCquery.query('SIarea > 0.12')
print(propOver)

                                  Depth    SIarea  iter          rA
YC         XC         time                                         
-77.157852 327.750000 2013-01-01  100.0  0.981079  1860  16961832.0
                      2013-01-02  100.0  0.979461  1920  16961832.0
                      2013-01-03  100.0  0.977427  1980  16961832.0
                      2013-01-04  100.0  0.975684  2040  16961832.0
                      2013-01-05  100.0  0.974274  2100  16961832.0
...                                 ...       ...   ...         ...
-76.895950 328.083344 2013-01-04  190.0  0.399158  2040  17648784.0
                      2013-01-05  190.0  0.156779  2100  17648784.0
           328.250000 2013-01-01  140.0  0.218696  1860  17648784.0
                      2013-01-02  140.0  0.365204  1920  17648784.0
                      2013-01-03  140.0  0.318326  1980  17648784.0

[188 rows x 4 columns]


In [20]:
proportionOverXpercent = propOver.shape[0] / XCquery.shape[0]
print(proportionOverXpercent)

0.05266106442577031


In [11]:
icePointsDF = icePointsDF.to_numpy()
print(icePointsDF)

[[0.00000000e+00 0.00000000e+00 1.86000000e+03 1.48840640e+07]
 [0.00000000e+00 0.00000000e+00 1.92000000e+03 1.48840640e+07]
 [0.00000000e+00 0.00000000e+00 1.98000000e+03 1.48840640e+07]
 ...
 [4.73689160e+03 0.00000000e+00 3.54000000e+03 2.92099872e+08]
 [4.73689160e+03 0.00000000e+00 3.60000000e+03 2.92099872e+08]
 [4.73689160e+03 0.00000000e+00 3.66000000e+03 2.92099872e+08]]


In [6]:
icePointsDF = icePointsDF['XC'].apply(lambda x: x if x <=180 else x - 360)
print(icePointsDF)

KeyError: 'XC'